In [ ]:
#default_exp learner

In [ ]:
#export
from mantisshrimp.imports import *
from mantisshrimp.core import *
from mantisshrimp.models import *

/home/lgvaz/anaconda3/envs/light/lib/python3.7/site-packages/graphql/type/directives.py:55: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  assert isinstance(locations, collections.Iterable), 'Must provide locations for directive.'
/home/lgvaz/anaconda3/envs/light/lib/python3.7/site-packages/graphql/type/typemap.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import OrderedDict, Sequence, defaultdict
/home/lgvaz/anaconda3/envs/light/lib/python3.7/site-packages/trains/backend_interface/metrics/events.py:27: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.attrs(cmp=False, slots=True)


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Learner" data-toc-modified-id="Learner-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Learner</a></span><ul class="toc-item"><li><span><a href="#Core" data-toc-modified-id="Core-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Core</a></span></li><li><span><a href="#Visualize" data-toc-modified-id="Visualize-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Visualize</a></span></li></ul></li><li><span><a href="#Export" data-toc-modified-id="Export-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Export</a></span></li></ul></div>

# Learner
> High level api for training models.

## Core

In [ ]:
#export
class MultiCycleLR(OneCycleLR):
    def load_state_dict(self, state_dict): pass

In [ ]:
#export
class Learner:
    def __init__(self, m, train_dl, valid_dl, opt_fn, logger=None, cbs=None):
        store_attr(self, 'm,train_dl,valid_dl,opt_fn')
        self.logger = logger or True
        self.gpus = get_all_available_gpus()
        self.cbs = L(cbs)
        
    @delegates(Trainer.__init__)
    def fit(self, max_epochs, lr, lr_sched_fn=None, gpus=None, callbacks=None, **kwargs):
        self.m.prepare_optimizers(self.opt_fn, lr, sched_fn=lr_sched_fn)
        gpus = ifnone(gpus, self.gpus)
        cbs = self.cbs + L(LearningRateLogger()) + L(callbacks)
        trainer = Trainer(max_epochs=max_epochs, logger=self.logger, callbacks=cbs,  gpus=gpus, **kwargs)
        trainer.fit(self.m, self.train_dl, self.valid_dl)
        
    @delegates(Trainer.__init__)
    def fit_one_cycle(self, max_epochs, lr_max, pct_start=.25, **kwargs):
        def lr_sched_fn(opt):
            lrs = self.m.get_lrs(lr_max)
            sched = MultiCycleLR(opt, lrs, len(self.train_dl)*max_epochs, pct_start=pct_start)
            return {'scheduler':sched, 'interval':'step'}
        return self.fit(max_epochs=max_epochs, lr=lr_max, lr_sched_fn=lr_sched_fn, **kwargs)
        
    @delegates(Trainer.__init__)
    def lr_find(self, gpus=None, **kwargs):
        self.m.prepare_optimizers(self.opt_fn, 0)
        gpus = ifnone(gpus, self.gpus)
        return Trainer(gpus=gpus, **kwargs).lr_find(self.m, self.train_dl, self.valid_dl)

## Visualize

In [ ]:
#export
@patch
def show_results(self:Learner, k=5):
    rs = random.choices(self.valid_dl.dataset.records, k=k)
    show_preds(*self.m.predict(rs=rs))

# Export

In [ ]:
from nbdev.export import notebook2script
notebook2script()

AssertionError: Use `create_config` to create settings.ini for the first time